<a href="https://colab.research.google.com/github/chengxuan-xia/TSP-Genetic-Algorithm/blob/main/TSP_Genetic_Algorithm_P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TSP Genetic Algorithm

## Problem Statement

You are given a list of destinations in Los Angeles along with their latitude and longitude (in degrees). Your goal is to visit all the destinations and return to your starting location efficiently (as defined by an objective function that we will specify and you will implement). The first location in the list read in from the text file is LAX, which will be your starting and finishing location.

You may recognize this problem setup as the Traveling Salesman Problem which is NP-Hard. You may be able to design an algorithm (i.e. brute force) for the list of locations we give you that computes the globally optimal solution and that has a tractible runtime. However, that algorithm won't scale.

Just because we can't find optimal solutions to problems like this doesn't mean we can disregard them. Real-world problems frequently map to setups like TSP, so different algorithmic strategies to tackle them have emerged, including genetic algorithms (also approximation algorithms, annealing, etc...)

Your challenge is to implement and assemble the components of a genetic algorithm which can be used to provide "good" albeit not optimal solutions for efficiently traversing our graph of Los Angeles.

## Genetic Algorithms Overview

Genetic Algorithms are a method for solving optimization problems based on natural selection. It tracks pretty closely to the tenets of Darwinism. In Darwinism, given some variation in genetic traits for some population, the members of that species with desirable genetic traits will survive more often and reproduce more, passing those genetics down, and thus, over generations, the desirable genetics will have larger market share.

The basic setup of a genetic algorithm is largely the same. Given some NP-hard problem, we don't assume to have any intuition about the "desirable traits" of a potential solution, so we generate some initial set of candidate solutions randomly. These solutions are a set of valid paths that begin at LAX, visit every node, and don't visit any node more than once*. 

This set of candidate solutions is called the **population**.

Each solution in the population is a **member**.

The number of members in a population is the **population size**.

Genetic algorithms undergo some specified number of iterations. Each iteration works with its own population (each with the same number of members). In the initial iteration, the members of a population are generated randomly.

Once you've generated your initial population, you need to evaluate the quality of each member (i.e. each candidate solution). This metric is called a member's **fitness**.

The more fit a member of a population is, the more likely it is to have substructure (i.e. genetic traits) that would improve other solutions (i.e. there may be a 4 node subsequence of that path that is very efficient even if it's a completely random shuffle).

Understanding the quality of each solution allows us to refine our strategy for generating new solutions. While the initial population was randomly generated, each member in a subsequent population is assembled as a composite of two members of the preceeding generation. Instead of choosing those two parents randomly, we can incorporate our grading of the current members fitnesses into the selection process and thus pass on preferable "genes". 

Genetic algorithms have heuristics for spawning the next generation's population from the current population:

1. Each member of the new generation is the offspring of two member's of the current population. 
2. More fit members of the current generation are more likely to be selected as parents for the new generation's population members
3. After both parents complete encoding a child, that child's genes (i.e. the new solution) may mutate. Mutation is a random event that modifies the child's genes ("solution"). In our case, mutation will simply be swapping the positions of two nodes in the child's path (i.e the child's solution) (the indices of those nodes will also be random).
4. When creating a child, each parent will pass on at least 1 element ("gene") of its solution, but there is no guarantee about the relative concentration of one parent's "genes" vs the other parent's "genes" in the offspring.**
5. Once both are selected, the relative fitness of one parent compared to the other has no weight in determining their share of the child's "genes"**
6. The order in which parents insert their "genes" into the child is random and also agnostic to their relative fitness**

The important concepts to understand are the first 3 items of that list. The exact mechanism through which two parents' solutions merge into a child's solution (1) will be explained when you go to implement it (our method isn't that complicated).

Mutation is a useful tactic both in evolution and in genetic algorithms. In evolution, mutation helps promote genetic diversity (i.e. so everyone doesn't converge to the same genetic makeup) and can introduce new traits that may be beneficial. In genetic algorithms, it serves an analogous purpose. Over generations, populations can start to converge, where members of a population can become similar and in turn produce a population of increasingly similar offspring in the next generation. If there's no countermeasure for this, your algorithm may converge to a local maxima before exploring much of the state space. Not only can mutation help prevent this, but the initial population that was randomly generated may be concentrated in a small portion of the state space. Injecting noise when creating subsequent populations can be beneficial in that case.

\* The last leg of any path is returning back to LAX but that isn't factored in until later (it'll make sense when you get to that point in the assignment). 

\** The last 3 points are implementation details more than conceptually important. It will make more sense once you get there

## Notebook Overview
- Run the first block to import several requirements for this project.
    - **typing** is used to provide type annotations to every function you are required to implement. Deviating from the input / output specifications will make it very difficult to then combine the various components at the end and should be avoided
    - **random** is the Python language module for random number generation and sampling distributions. We pass a seed to the random number generator so that we can get consistent results and replicate your work. random defaults to the system time without a seed. **Please don't reseed random**
    - **unittest** is used to provide a couple unit tests that you can use to check your implementation before integrating it into larger functions. The more substantial, top-level functions don't include unit tests because those are unwieldy to write.
    - **utils** includes several class definitions and utility functions we provide for you to use
- You can view the implementations of Location, FitnessPair, and ParentPair in **utils.py**. You will use all three so you need to be familiar with them

In [ ]:
!wget -q "https://drive.google.com/uc?export=download&id=1MJAG0spB3vlOF7-0C-G9Zl2SI8qPWl4U" -O GA_startercode.zip
!unzip -qq GA_startercode.zip

In [ ]:
from typing import List
import random
import unittest

from utils import Location, FitnessPair, ParentPair

random.seed(42)

## Algorithm Parameters

There are 3 major configurable parameters for this algorithm:

1. **num_generations** is the number of generations (i.e. number of iterations the algorithm executes, each time creating a new population of candidate solutions). Note: num_generations doesn't count generating the initial population randomly as an iteration
2. **population_size** is the size of the population (i.e. number of candidate solutions every generation)
3. **mutation_probability** is the probability that a newly created child will be mutated (no more than once per child)

**filename** is the relative path to the file with the list of locations. It's a headerless csv file that follows the pattern:

Name, Latitude, Longitude

with latitude and longitude given in degrees, not radians.

If you want to swap in your own file, make sure to follow that convention.

In [ ]:
population_size = 10
num_generations = 200
mutation_probability = 0.6
filename = './input/locations.txt'
solution, score = [], None

## Setup

The below cell reads in the locations from the file with the path *filename* and then generates an initial population of size *population_size*. **population** is a list of lists, with each inner list providing a valid solution, traversing all the locations. The path is given as a series of integers, which correspond to indexes in the locations list 

In [ ]:
from utils import generate_initial_population, parse_locations

# you can print both of these for clarity if you want

locations = parse_locations(filename) # List[Location]

population = generate_initial_population(population_size, len(locations)) # List[List[int]]

In [ ]:
population

[[0, 15, 14, 5, 10, 6, 16, 18, 7, 13, 17, 11, 2, 12, 3, 19, 8, 9, 1, 4],
 [0, 15, 9, 4, 12, 11, 17, 2, 13, 6, 19, 3, 1, 5, 10, 16, 8, 14, 7, 18],
 [0, 15, 14, 19, 6, 11, 3, 5, 10, 7, 2, 16, 8, 17, 1, 13, 9, 18, 12, 4],
 [0, 1, 13, 16, 11, 9, 19, 12, 6, 17, 5, 7, 15, 4, 14, 18, 10, 8, 2, 3],
 [0, 13, 12, 14, 10, 19, 15, 2, 1, 16, 4, 9, 11, 5, 7, 17, 18, 8, 6, 3],
 [0, 10, 1, 2, 15, 9, 13, 6, 14, 4, 12, 5, 3, 8, 18, 17, 16, 19, 11, 7],
 [0, 14, 8, 10, 6, 19, 9, 4, 15, 7, 12, 11, 18, 2, 1, 13, 3, 16, 17, 5],
 [0, 17, 10, 14, 12, 16, 15, 19, 8, 3, 7, 5, 2, 6, 11, 13, 9, 4, 1, 18],
 [0, 2, 4, 11, 16, 14, 18, 19, 8, 15, 1, 12, 9, 3, 13, 6, 5, 7, 17, 10],
 [0, 15, 7, 1, 4, 6, 5, 12, 10, 19, 11, 17, 9, 13, 2, 14, 16, 18, 3, 8]]

In [ ]:
locations

[Name: LAX Airport, Latitude: 33.941845, Longitude: -118.408635,
 Name: Tommy Trojan, Latitude: 34.020547, Longitude: -118.285397,
 Name: Coliseum, Latitude: 34.014156, Longitude: -118.287923,
 Name: Chinese Theatre, Latitude: 34.102021, Longitude: -118.340946,
 Name: Whiskey a Go Go, Latitude: 34.090839, Longitude: -118.385725,
 Name: Getty Center, Latitude: 34.078062, Longitude: -118.473892,
 Name: Getty Villa, Latitude: 34.045868, Longitude: -118.56485,
 Name: Disneyland, Latitude: 33.81211, Longitude: -117.918921,
 Name: The Huntington Library, Latitude: 34.129178, Longitude: -118.114556,
 Name: Rose Bowl, Latitude: 34.161373, Longitude: -118.167646,
 Name: Griffith Observatory, Latitude: 34.118509, Longitude: -118.300414,
 Name: Hollywood Sign, Latitude: 34.134124, Longitude: -118.321548,
 Name: Magic Mountain, Latitude: 34.425392, Longitude: -118.59723,
 Name: Third Street Promenade, Latitude: 34.016297, Longitude: -118.496838,
 Name: Venice Beach, Latitude: 33.985857, Longitude:

## Fitness

For fitness, we will use the Haversine distance formula to compute the distance between each step on the path:

$ 𝛿 lon = lon_{2} - lon_{1} $

$ 𝛿 lat = lat_{2} - lat_{1} $

$ a = (sin(\frac{𝛿 lat}{2}))^{2} + cos(lat_{1}) cos(lat_{2}) (sin(\frac{𝛿 lon}{2}))^{2} $

$ c = 2 \times atan2(\sqrt{a}, \sqrt{1-a}) $

$ distance = 3961 c $

**Important Note**: The latitude and longitude values provided by the input file are in degrees, not radians, but this formula requires radians. The convenience function *convert_to_radians* is provided to you. Refer to **utils.py** for details on usage

In [ ]:
from utils import convert_to_radians
import math # Will be required to implement calculate_haversine_distance
from math import cos, sin, asin, sqrt, atan2

def calculate_haversine_distance(loc_1: Location, loc_2: Location) -> float:
    rlon = convert_to_radians(loc_2).longitude - convert_to_radians(loc_1).longitude
    rlat = convert_to_radians(loc_2).latitude - convert_to_radians(loc_1).latitude
    a = sin(rlat/2)**2 + cos(convert_to_radians(loc_1).latitude)*cos(convert_to_radians(loc_2).latitude)*sin(rlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return 3961 * c
    # pass

In [ ]:
calculate_haversine_distance(locations[1],locations[0])

8.917052227102195

In [ ]:
# test only
# convert_to_radians(locations[0])

In [ ]:
# test only
# a = convert_to_radians(locations[0])
# a.longitude

You can verify your implementation of the haversine distance function by running the cell below

In [ ]:
class TestHaversine(unittest.TestCase):
    def setUp(self):
        self.location_1 = Location("test1", 40.0, 50.0)
        self.location_2 = Location("test2", 20.0, 30.0)
        
    def test_calculate_haversine_distance_same_location(self):
        self.assertEqual(0.0, calculate_haversine_distance(self.location_1, self.location_1))

    def test_calculate_haversine_distance_different_locations(self):
        answer = calculate_haversine_distance(self.location_1, self.location_2)
        self.assertEqual(1820.0246, round(answer, 4))
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.006s

OK


Calculate the fitness of a population member by computing the haversine distance between each stop on the path. Don't forget to include the distance from the last location on the path back to LAX to complete the cycle

In [ ]:
def calculate_member_fitness(pop_member: List[int], locs: List[Location]) -> float:
    member_fitness = 0
    for mb in range(len(pop_member)):
      if mb == len(pop_member) - 1:
        distance = calculate_haversine_distance(locs[pop_member[mb]], locs[0])
      else:
        distance = calculate_haversine_distance(locs[pop_member[mb]], locs[mb+1])
      member_fitness += distance
    return member_fitness
    # pass

In [ ]:
class TestMemberFitness(unittest.TestCase):
    def setUp(self):
        self.locations = parse_locations('./input/test_locs.txt')
        self.member = [0,1,2,3,4,5]
        
    def test_calculate_member_fitness(self):
        self.assertEqual(77.4213, round(calculate_member_fitness(self.member, self.locations), 4))
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.012s

OK


Implement the following function which should compute the fitness of each member of the population. 

The output should be a list of FitnessPairs. The FitnessPair id corresponds to the index of that member in the population list, and the FitnessPair fitness is that member's fitness score. We use this data structure because later we will sort by fitness but you'll still need to know which member of the population a score corresponds to.

Before you return this list, sort it so that FitnessPair with the best fitness score is at the front. Remember that a **low** fitness score is a good one, so you'd return a list with scores in **ascending** order

In [ ]:
def calculate_population_fitness(population: List[List[int]], locs: List[Location]) -> List[FitnessPair]:
    population_fitness = []
    for member_id in range(len(population)):
      member_fitness = calculate_member_fitness(population[member_id], locs)
      population_fitness.append(FitnessPair(member_id, member_fitness))
    return sorted(population_fitness, key = lambda FitnessPair: FitnessPair.fitness, reverse = False)
    # pass

In [ ]:
a = calculate_population_fitness(population, locations)
a

[Id: 4, Fitness: 297.88677770280134,
 Id: 7, Fitness: 345.6225291518224,
 Id: 8, Fitness: 352.9855024578175,
 Id: 9, Fitness: 369.9335195442553,
 Id: 6, Fitness: 370.58115343996417,
 Id: 0, Fitness: 378.25001548380135,
 Id: 1, Fitness: 384.1362214404404,
 Id: 2, Fitness: 385.6391779483609,
 Id: 3, Fitness: 404.9294804256869,
 Id: 5, Fitness: 412.6734954546638]

## Selection

Now that we've computed the fitness of each member of the population and sorted the results, we can implement a strategy for selecting parents with better genetics (i.e. solutions) to produce more "fit" children in the next generation.

Parent selection is probabilistic, but we will tilt the odds to favor the fitter members of the population. 

The following function, *generate_fitness_probability_vector*, takes the list returned from the previous function and uses it to assign a probability to each population member (the probability they will be selected on any given parent selection event). 

Implement the following:
1. Create a list with size n = # of population members and assign each element in the list to 1.0/n, giving a uniform distribution
2. Note that the indices in this vector should correspond to the individual’s population index number (so this most will likely not be sorted by fitness). The first element corresponds to the first population member, not the member with the best fitness score (unless that is the first population member)
3. The two individuals with the highest fitness scores should have their probability multiplied by 6.0.
4. The remainder of the top half of the fit individuals (eg. from rank 2 to rank size / 2 – 1) should have their probability multiplied by 3.0.
5. Note that step 4 is rounding down. For example, if the population had 7 members, the two most fit would be multiplied by 6 and the third most fit multiplied by 3 BUT NOT THE FOURTH
6. Renormalize the probability vector to sum to 1
7. Return the normalized vector (referred to subsequently as **fitness_probs**)

In [ ]:
def generate_fitness_probability_vector(population_fitness: List[FitnessPair]) -> List[float]:
    half = len(population_fitness) // 2
    unit = 1.0 / len(population_fitness)
    prob_vec = []
    for order in range(len(population_fitness)):
      if order + 1 <= 2:
        prob = float(6.0 * unit )
      elif order + 1 <= half:
        prob = float(3.0 * unit )
      else:
        prob = unit
      prob_vec.insert(population_fitness[order].id, prob)
    prob_sum = sum(prob_vec)
    prob_vec_norm = [float(i/prob_sum) for i in prob_vec]
    return prob_vec_norm
    # pass

You can verify your implementation of the generate_fitness_probability_vector function by running the cell below

In [ ]:
class TestFitnessProbabilityVector(unittest.TestCase):
    def setUp(self):
        self.test_input = []
        self.test_input.append(FitnessPair(2,  60.02))
        self.test_input.append(FitnessPair(1,  60.23))
        self.test_input.append(FitnessPair(0,  63.90))
        self.test_input.append(FitnessPair(5,  77.38))
        self.test_input.append(FitnessPair(4,  78.22))
        self.test_input.append(FitnessPair(3,  81.43))
        self.ground_truth = [0.167, 0.333, 0.333, 0.056, 0.056, 0.056]
        
    def test_generate_fitness_probability_vector_(self):
        answer = generate_fitness_probability_vector(self.test_input)
        for i in range(6):
            self.assertEqual(self.ground_truth[i], round(answer[i], 3))
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.016s

OK


Now we will use this distribution, **fitness_probs**, to select a single parent. First, lets consider this hypothetical input vector:

[.1, .4, .1, .4]

What this input translates to is that the population member at index 0 has a 10% chance of being selected, the population member at index 1 has a 40% chance of being selected, the population member at index 2 has a 10% chance of being selected, and the population member at index 3 has a 40% chance of being selected.

**select_parent** will randomly select an index, and the population member at that index will be the selected parent.

First, select_parent should generate a random float between 0 and 1

The algorithm for picking an index is simple. Starting at index 0, you perform a running sum of
the probability. If at any index, the running sum becomes >= the random float, you select that
individual.

For example, suppose the random double is 0.55. So:
- At index 0, running sum = 0.10, continue
- At index 1, running sum = 0.50, continue
- At index 2, running sum = 0.60, >= 0.55, so select index 2

So in the above example, the following ranges of random values would select each individual:
- Individual 0 = [0, 0.10]
- Individual 1 = (0.10, 0.50]
- Individual 2 = (0.50, 0.60]
- Individual 3 = (0.60, 1.00]

In [ ]:
def select_parent(fitness_probs: List[float]) -> int:
    chance = random.random()
    running_sum = 0
    for index in range(len(fitness_probs)):
      running_sum += fitness_probs[index]
      if running_sum >= chance:
        return index
    # pass

Taking the output from *generate_fitness_probability_vector*, **fitness_probs**, as input, we need to select *n* pairs of parents, where *n* is the size of the population. These *n* pairs of parents will then be used to create the next generation's *n* population members.

This function should return a list of ParentPair (a class defined in utils). Each parent in a pair should be selected independently (i.e. using its own random float threshold.

In [ ]:
def select_parents(fitness_probs: List[float]) -> List[ParentPair]:
    parents = []
    for i in range(len(fitness_probs)):
      p1 = select_parent(fitness_probs)
      p2 = select_parent(fitness_probs)
      parents.append(ParentPair(p1, p2))
    return parents
    # pass

In [ ]:
a = select_parents([0.167, 0.333, 0.333, 0.056, 0.056, 0.056])
a

[Parents: 2, 1,
 Parents: 2, 1,
 Parents: 4, 1,
 Parents: 1, 0,
 Parents: 0, 2,
 Parents: 2, 0]

## Crossover

The crossover function takes two parent population members (each is a list of ints from the population 2d list) and create a child population member. The float mutation_prob [0.0,1.0] is the probability of doing a mutation on that child population member.

Implement the following crossover procedure:
1. Generate a random crossover index value from [1, size –2]
2. Generate a random 0 or 1 (simulate a coin toss). If it’s a 1, parent_pop_1 goes first. If it’s a 0, parent_pop_2 goes first.
3. Selected first parent will copy all elements from beginning up to and including crossover index into child
4. Second parent will start at the beginning (of its list, not the childs), and append all elements that don’t already appear in the child
5. Generate a random float, *x*, between 0 and 1
6. If *x* <= mutation_prob, then mutate. If *x* > mutation_prob, don't mutate

Implement the following mutation procedure:
1. Generate a random int *a* [1, size-1]
2. Generate a random int *b* [1, size-1]
3. Swap the elements at those two indices in the child population

**Note**: You may find it helpful to add a helper function or two to avoid repeated code / improve readability. If you choose to do so, you can either create additional cells or include them in the **crossover** cell

In [ ]:
def mutate(child_pop: List[int], mutation_prob:float):
    random_mutat = random.random()
    if random_mutat <= mutation_prob:
      int1 = 1 + int(random.random() * (len(child_pop)-1))
      int2 = 1 + int(random.random() * (len(child_pop)-1)) # could be the same?
      pop1 = child_pop[int1]
      pop2 = child_pop[int2]
      child_pop[int1] = pop2
      child_pop[int2] = pop1
    return child_pop

In [ ]:
def parents_cross(parent_pop_1: List[int], parent_pop_2: List[int], crossover_index: int)-> List[int]:
    child = []
    for i in range(0, crossover_index+1):
      child.append(parent_pop_1[i])
    for j in parent_pop_2:
      if j not in child:
        child.append(j)
    return child

In [ ]:
def crossover(parent_pop_1: List[int], parent_pop_2: List[int], mutation_prob: float) -> List[int]:
    coin = random.sample([0, 1], 1)[0]
    index_ls = [i for i in range(1, len(parent_pop_1)-1)]
    crossover_index = random.sample(index_ls, 1)[0]
    if coin:
      child_pop = parents_cross(parent_pop_1, parent_pop_2, crossover_index)
    else:
      child_pop = parents_cross(parent_pop_2, parent_pop_1, crossover_index)
    child_pop_mut = mutate(child_pop, mutation_prob)
    return child_pop_mut
    # pass


The generate_new_population function takes the current population, the list of parent pairs, and the mutation probability as input to generate a new population for the next generation of the algorithm. It does so by iterating over the list of parent pairs, looking up their corresponding entries in the population, and then spawning a child population member with crossover

In [ ]:
def generate_new_population(population: List[List[int]], parent_selections: List[ParentPair], mutation_prob: float) -> List[List[int]]:
    new_population = []
    for parents in parent_selections:
      p1 = parents.first
      p2 = parents.second
      child_pop = crossover(population[p1], population[p2], mutation_prob)
      new_population.append(child_pop)
    return new_population
    # pass

## Genetic Algorithm

Using some subset of the functions you've implemented above, fill in the parts of the algorithm marked TODO.

**Note**: Make sure that you've run all previous cells before running the below cell to ensure that every function the genetic algorithm needs is available

In [ ]:
from utils import validate_solution

num_generations = 1000
mutation_probability = 0.1

population = generate_initial_population(population_size, len(locations)) # List[List[int]]
for i in range(num_generations+1):
    # TODO: Calculate population fitnesses
    population_fitness = calculate_population_fitness(population, locations)
    # continue iterating with GA
    if i != num_generations:
        scores_list = calculate_population_fitness(population, locations)
        # print(scores_list)
        # TODO: Get fitness probability vectors
        fitness_probs = generate_fitness_probability_vector(population_fitness)
        # TODO: Get parent selections
        parent_selections = select_parents(fitness_probs)
        # TODO: Generate new population
        population = generate_new_population(population, parent_selections, mutation_probability)
        
    else:
        # Choose the top result in the population fitnesses list as your solution
        # TODO: Retrieve that solution from the population along with its fitness score
        scores_list = calculate_population_fitness(population, locations)
        # print(scores_list)
        # solution, score = population[scores_list[0].id], scores_list[0].fitness
        solution = population[scores_list[0].id]
        score = calculate_member_fitness(solution, locations)
        score1 = scores_list[0].fitness
        print(solution)
        print(score, score1)
        # Check if solution is valid
        is_valid_solution = validate_solution(solution, score, locations)
        if is_valid_solution:
            print('Success!')
            print('Score: ', score)
            solution_locs = [locations[i] for i in solution]
            print('Solution: ', solution_locs)
        else:
            print('Solution invalid')

[0, 9, 3, 4, 5, 11, 7, 6, 8, 16, 18, 12, 10, 1, 15, 2, 17, 13, 19, 14]
117.53738796432773 117.53738796432773
Fitness score computed incorrectly
Solution invalid
